In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd 
from keras.layers import LSTM,Dropout,Dense
from keras.layers import SimpleRNN
from keras.layers import GRU
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Conv1D, Flatten, Dropout, LSTM, Reshape,MaxPooling1D
from tensorflow.keras.models import Model

**Read data**


In [12]:
import pandas as pd
df = pd.read_csv("/Users/rahuljauhari/Desktop/research-runoff/Data/merged_imd.csv")
df.drop(columns=['Unnamed: 0'], inplace=True)
df = df.iloc[:, :157]

df['DateTime'] = pd.to_datetime(df['DateTime'])
df.set_index('DateTime', inplace=True)
monthly_mean = df.resample('M').mean()
print(monthly_mean.shape)

df_actual = pd.read_excel("/Users/rahuljauhari/Desktop/research-runoff/Data/Calibrated and Validated.xlsx")
# select last column
observed_runnoff = df_actual['observed']
# observed_runnoff.head()
print(observed_runnoff.shape)

(468, 156)
(468,)


In [13]:
from scipy.stats import zscore
def func(name):
    x=0
    y=0
    inv= 0
    if name=='zscore':
        x_norm = zscore(monthly_mean)
        y_norm = zscore(observed_runnoff)
        x_norm[x_norm > 3] = 2.8
        x_norm[x_norm < -3] = -2.8
        y_norm[y_norm >3] = 2.8
        y_norm[y_norm < -3] = -2.8
        x=x_norm
        y=y_norm
    if name=='StandardScaler':
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(monthly_mean)
        y_scaled = scaler.fit_transform(observed_runnoff.values.reshape(-1,1))
        x_scaled[x_scaled > 3] = 2.8
        x_scaled[x_scaled < -3] = -2.8
        y_scaled[y_scaled >3] = 2.8
        y_scaled[y_scaled < -3] = -2.8
        x=      x_scaled  
        y=y_scaled
        inv = scaler
        
    if name == 'MinMaxScaler':
        scaler = MinMaxScaler(feature_range=(0,1))
        x_scaled = scaler.fit_transform(monthly_mean)
        y_scaled = scaler.fit_transform(observed_runnoff.values.reshape(-1,1))
        x=      x_scaled  
        y=y_scaled
        inv = scaler
    return x,y,inv


**Metric**

In [14]:
from sklearn.metrics import mean_squared_error
def rmse1(yt, yp): #lower the better
    return np.sqrt(mean_squared_error(yt, yp))
# Kling-Gupta effciency
def kge1(yt, yp): #highqer the better
    r = np.corrcoef(yt, yp,rowvar=False)[0, 1]
    alpha = np.std(yp) / np.std(yt)
    beta = np.mean(yp) / np.mean(yt)
    return 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
# Normalized standard Error 
def nse1(yt, yp): 
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)
    # r squared
def r21(yt, yp): #higher the better
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)

In [15]:
# activation = ['relu','sigmoid','tanh']
# optimizer =['adam','rmsprop']
# preprocess = ['StandardScaler']
activation = ['tanh']
optimizer =['adam']
preprocess = ['StandardScaler']

In [16]:
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
for act in activation:
    for opt in optimizer:
        for pre in preprocess:
            x,y,inv_scaler= func(pre)
            X_train, X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,shuffle=False)
            
            inputs = Input(shape=(156, 1))
            x = Conv1D(filters=64, kernel_size=3, activation=act, trainable=True, padding='same')(inputs)
            x = MaxPooling1D((2), strides=(1), padding='same')(x)
            
            # x = Conv1D(filters=64, kernel_size=3, activation=act, trainable=True, padding='same')(x)
            # x = MaxPooling1D((2), strides=(1), padding='same')(x)

            x = Flatten(name="flatten")(x)
            x = Dropout(0.5)(x)

            x = Reshape((156, 64))(x)  
            x = GRU(64, activation=act,return_sequences=True)(x)
            x = Dropout(0.2)(x) 
            
            x = LSTM(64, activation=act)(x)
            x = Dropout(0.2)(x) 
            
            output = Dense(1, activation=act)(x)

            model = Model(inputs=inputs, outputs=output)

            model.compile(optimizer=opt, loss='mse')
            model.fit(X_train, y_train, batch_size=100, epochs=100,shuffle=False, use_multiprocessing=True,verbose=0)

            y_pred_train = model.predict(X_train)
            y_pred_test = model.predict(X_test)
            y_pred_train_inv = inv_scaler.inverse_transform(y_pred_train)
            y_pred_test_inv = inv_scaler.inverse_transform(y_pred_test)
            y_train__inv = observed_runnoff[:len(y_pred_train_inv)]
            y_test__inv = observed_runnoff[len(y_pred_train_inv):]
            try:
                _ = pd.DataFrame({'pre':pre,'act':act,'opt':opt,
                                'KGE train':round(kge1(y_train__inv, y_pred_train_inv),4),'KGE test':round(kge1(y_test__inv, y_pred_test_inv),4)},index=[0])
                _.to_csv(os.getcwd()+'/imd_cnn_gru_lstm.csv',mode='a',header=True)
            except Exception as e:
                print(e)

3/3 [==============================] - 0s 31ms/step


In [17]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
y_pred_train_inv = inv_scaler.inverse_transform(y_pred_train)
y_pred_test_inv = inv_scaler.inverse_transform(y_pred_test)
y_train__inv = observed_runnoff[:len(y_pred_train_inv)]
y_test__inv = observed_runnoff[len(y_pred_train_inv):]
print("KGE train: ", round(kge1(y_train__inv, y_pred_train_inv),4))
print("KGE test: ", round(kge1(y_test__inv, y_pred_test_inv),4))



3/3 [==============================] - 0s 31ms/step
KGE train:  0.5295
KGE test:  0.5084


In [18]:
y_train__inv = pd.DataFrame(y_train__inv)
y_pred_train_inv = pd.DataFrame(y_pred_train_inv)
y_train__inv.reset_index(drop=True, inplace=True)
y_pred_train_inv.reset_index(drop=True, inplace=True)
y_train__inv = pd.concat([y_train__inv,y_pred_train_inv],axis=1)
y_train__inv.to_csv(os.getcwd()+'train_cnn_gru_lstm_0.2.csv',mode='a',header=True)
y_test__inv = pd.DataFrame(y_test__inv)
y_pred_test_inv = pd.DataFrame(y_pred_test_inv)
y_test__inv.reset_index(drop=True, inplace=True)
y_pred_test_inv.reset_index(drop=True, inplace=True)
y_test__inv = pd.concat([y_test__inv,y_pred_test_inv],axis=1)
y_test__inv.to_csv(os.getcwd()+'test_cnn_gru_lstm_0.2.csv',mode='a',header=True)

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
def func(name,x,y):
    inv = 0

    if name == 'StandardScaler':
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(x)
        y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))
        x = x_scaled
        y = y_scaled
        inv = scaler

    if name == 'MinMaxScaler':
        scaler = MinMaxScaler(feature_range=(0, 1))
        x_scaled = scaler.fit_transform(x)
        y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))
        x = x_scaled
        y = y_scaled
        inv = scaler
        
    return x, y, inv

In [24]:
df_ssp = pd.read_csv('/Users/rahuljauhari/Desktop/research-runoff/Data/SSP_monthly_585.csv')
df_ssp['TimeDate'] = pd.to_datetime(df_ssp['TimeDate'])
df_ssp.set_index('TimeDate', inplace=True)
monthly_mean = df_ssp
print(monthly_mean.shape)
scaler = StandardScaler()
x,y,inv_scaler= func('StandardScaler',monthly_mean,observed_runnoff)
# y=scaler.fit_transform(observed_runnoff.values.reshape(-1,1))
print(x.shape)
y_pred = model.predict(x)
print(y_pred.shape)
y_inv = inv_scaler.inverse_transform(y_pred)
pd.DataFrame(y_inv).to_csv('SSP_monthly_585_cnn_gru_lstm_0.2.csv',header=True)

df_ssp = pd.read_csv('/Users/rahuljauhari/Desktop/research-runoff/Data/SSP_monthly_370.csv')
df_ssp['TimeDate'] = pd.to_datetime(df_ssp['TimeDate'])
df_ssp.set_index('TimeDate', inplace=True)
monthly_mean = df_ssp
print(monthly_mean.shape)
x,y,inv_scaler= func('StandardScaler',monthly_mean,observed_runnoff)

print(x.shape)
y_pred = model.predict(x)
print(y_pred.shape)
y_inv = inv_scaler.inverse_transform(y_pred)
pd.DataFrame(y_inv).to_csv('SSP_monthly_370_cnn_gru_lstm_0.2.csv',header=True)

df_ssp = pd.read_csv('/Users/rahuljauhari/Desktop/research-runoff/Data/SSP_monthly_245.csv')
df_ssp['DateTime'] = pd.to_datetime(df_ssp['DateTime'])
df_ssp.set_index('DateTime', inplace=True)
monthly_mean = df_ssp.resample('M').mean()
print(monthly_mean.shape)
x,y,inv_scaler= func('StandardScaler',monthly_mean,observed_runnoff)

print(x.shape)
y_pred = model.predict(x)
print(y_pred.shape)
y_inv1 = inv_scaler.inverse_transform(y_pred)
pd.DataFrame(y_inv1).to_csv('SSP_monthly_245_cnn_gru_lstm_0.2.csv',header=True)

df_ssp = pd.read_csv('/Users/rahuljauhari/Desktop/research-runoff/Data/SSP_monthly_126.csv')
df_ssp['DateTime'] = pd.to_datetime(df_ssp['DateTime'])
df_ssp.set_index('DateTime', inplace=True)
monthly_mean = df_ssp
print(monthly_mean.shape)
x,y,inv_scaler= func('StandardScaler',monthly_mean,observed_runnoff)

print(x.shape)
y_pred = model.predict(x)
print(y_pred.shape)
y_inv = inv_scaler.inverse_transform(y_pred)
pd.DataFrame(y_inv).to_csv('SSP_monthly_126_cnn_gru_lstm_0.2.csv',header=True)


(948, 156)
(948, 156)
30/30 [==============================] - 1s 32ms/step
(948, 1)
(948, 156)
(948, 156)
30/30 [==============================] - 1s 36ms/step
(948, 1)
(948, 156)
(948, 156)
30/30 [==============================] - 1s 34ms/step
(948, 1)
(948, 156)
(948, 156)
30/30 [==============================] - 1s 32ms/step
(948, 1)
